In [ ]:
#Advent of Code Problem 4
#...I dont know why I wasnt using functions before...
#of course we now have two data type in one input....

import requests
import numpy as np


def getPuzzleInputAsList(day):
    cookie_file = open("../keys/session.cookie")
    cookie_raw = cookie_file.readline()
    cookie = cookie_raw.strip("\n")
    sesh = requests.Session()
    url = "https://adventofcode.com/2021/day/"+str(day)+"/input"
    req = sesh.get(url,cookies={"session":cookie})
    input_raw = req.text
    input_list = input_raw.split("\n")
    return input_list

#parse winning numbers out into a flat array
def getWinningNums(input_list):
    raw_winning = input_list[0]
    winning = raw_winning.split(",")
    return np.array(winning,dtype=np.int64)

#parse boards into a 3d shape
def getGameBoards(input_list):
    raw_gameboards = np.delete(input_list,np.argwhere(input_list == '')) #litty: https://thispointer.com/delete-elements-from-a-numpy-array-by-value-or-conditions-in-python/
    gameboards = np.array([],dtype=np.int64)
    for ea in raw_gameboards:
        gameboard_spaces = np.array(ea.split(" "))
        gameboard_num = np.array(np.delete(gameboard_spaces,np.argwhere(gameboard_spaces == '')),dtype=np.int64)
        gameboards = np.append(gameboards,gameboard_num)
    
    gameboards = np.reshape(gameboards,(-1,5,5))
    return gameboards

#data was coming in as string so this wasnt working, parsed data to int during GB creation; works on entire array now
def replaceWinner(gameboard,winner):
    gameboard = np.where(gameboard == winner, 0, gameboard)
    return gameboard

#https://stackoverflow.com/questions/69601220/bingo-using-python
#tests for lines of 0's in bingo card
def isWinner(gameboard):
    for i in range (5):
        row_zeros=np.count_nonzero(gameboard[i:,])#would like to understand this : notation
        col_zeros=np.count_nonzero(gameboard[:,i])
        if not row_zeros or not col_zeros: #check if we have 0 non-zeros, if line with non-zeros is discovered continue
            return (True)
    diagonal_zeros=np.count_nonzero(np.diag(gameboard))
    diagonal1_zeros=np.count_nonzero(np.diag(np.fliplr(gameboard)))
    if not diagonal_zeros or not diagonal1_zeros:
        return(True)
    return(False)


def bingoSubsystem(gameboards,numbers):
    winner = False
    winning_board = np.array([])
    for ea in winning_nums:
        win_num = ea
        if winner:
            break
        board_counter = 0
        gameboards = replaceWinner(gameboards,win_num)
        for ea in gameboards:
            winner = isWinner(ea)
            if winner:
                winning_board_index = board_counter
                winning_board = ea
                return win_num,winning_board_index,winning_board
            board_counter+=1
    return "No winning board??"

input_list = getPuzzleInputAsList(4) # this is done to reduce stress on AoC servers, only one call is needed

gameboards = getGameBoards(np.delete(input_list,0))#gets game boards after removing winning numbers from input_list

winning_nums = getWinningNums(input_list)

winning_details = bingoSubsystem(gameboards,winning_nums)
raw_winning_board = gameboards[winning_details[1]]
marked_winning_board = winning_details[2]
winning_number = winning_details[0]
print(winning_number)
print(raw_winning_board)
print(marked_winning_board)
print(winning_number*np.sum(marked_winning_board))